In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning
import warnings

# Ignorer les avertissements de convergence pour plus de clarté
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning) # Pour certains warnings de scikit-learn

# --- 1. Charger les données ---
try:
    # Assure-toi que le fichier CSV est dans le même répertoire que ton notebook,
    # ou ajuste le chemin d'accès.
    df = pd.read_csv('electric_vehicle_analytics.csv', delimiter=',') # 
    print("Données chargées avec succès !")
    print(f"Dimensions du DataFrame : {df.shape}")
except FileNotFoundError:
    print("Erreur : Le fichier 'electric_vehicle_analytics.csv' n'a pas été trouvé.")
    exit() # Arrête le script si le fichier n'est pas trouvé

# Afficher les premières lignes et les informations générales
print("\n--- Aperçu des données ---")
print(df.head())
print("\n--- Informations sur les colonnes et types de données ---")
df.info()

# --- 2. Analyse Exploratoire (focus sur la cible) ---
print("\n--- Statistiques descriptives de la variable cible (Range_km) ---")
print(df['Range_km'].describe())

print("\n--- Visualisation de la distribution de la variable cible (Range_km) ---")
plt.figure(figsize=(10, 6))
sns.histplot(df['Range_km'], kde=True)
plt.title('Distribution de l\'autonomie (Range_km)')
plt.xlabel('Autonomie (km)')
plt.ylabel('Fréquence')
plt.grid(axis='y', alpha=0.5)
plt.show()
# Commentaire : Observer la forme de la distribution (symétrie, asymétrie, pics, etc.)

# --- 3. Prétraitement des données ---
print("\n--- Prétraitement des données ---")

# Séparer les caractéristiques (X) et la cible (y)
X = df.drop('Range_km', axis=1)
y = df['Range_km']

# Identifier les colonnes numériques et catégorielles (exclure l'ID et potentiellement Model)
# Note: On exclut 'Model' car il a trop de valeurs uniques pour OHE simple.
# Tu pourrais explorer d'autres techniques d'encodage si 'Model' est important.
numerical_features = X.select_dtypes(include=np.number).drop('Vehicle_ID', axis=1, errors='ignore').columns.tolist()
categorical_features = ['Make', 'Region', 'Vehicle_Type', 'Usage_Type'] # Exclut Model pour cet exemple

print(f"Caractéristiques numériques sélectionnées : {numerical_features}")
print(f"Caractéristiques catégorielles sélectionnées : {categorical_features}")

# Créer les transformateurs
# Mise à l'échelle pour les numériques, One-Hot Encoding pour les catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # ignore les catégories non vues à l'entraînement
    ],
    remainder='drop' # Ignore les autres colonnes (comme 'Model')
)

# --- 4. Séparation Entraînement / Test ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} échantillons")

# --- 5. Création et Entraînement du Pipeline (Modèle RandomForestRegressor) ---
# Utilisation d'un pipeline pour intégrer le prétraitement et le modèle
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))]) # n_jobs=-1 utilise tous les processeurs

print("\n--- Entraînement du modèle RandomForestRegressor ---")
model_pipeline.fit(X_train, y_train)
print("Entraînement terminé.")

# --- 6. Évaluation sur l'ensemble de test ---
print("\n--- Évaluation sur l'ensemble de test ---")
y_pred = model_pipeline.predict(X_test)

# Calcul des métriques de régression
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f} km")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} km")
print(f"Coefficient de détermination (R²): {r2:.3f}")
# Commentaire : MAE/RMSE donnent l'erreur moyenne en km. R² indique la part de variance expliquée.

# --- 7. Validation Croisée (KFold) ---
print("\n--- Validation Croisée (KFold) ---")
# Définir la stratégie de validation croisée (KFold pour la régression)
cv = KFold(n_splits=5, shuffle=True, random_state=42) #  suggère StratifiedKFold mais KFold est plus commun pour la régression

# Calculer les scores en utilisant R² comme métrique
# Note: cross_val_score utilise par défaut le score R² pour les régresseurs
cv_scores_r2 = cross_val_score(model_pipeline, X, y, cv=cv, scoring='r2', n_jobs=-1)

# Calculer les scores en utilisant l'erreur absolue moyenne négative (plus proche de 0 est mieux)
cv_scores_mae = cross_val_score(model_pipeline, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)

print(f"Scores R² de la validation croisée : {np.round(cv_scores_r2, 3)}")
print(f"Moyenne R² de la validation croisée : {cv_scores_r2.mean():.3f}")
print(f"Scores MAE (négatifs) de la validation croisée : {np.round(cv_scores_mae, 2)}")
print(f"Moyenne MAE de la validation croisée : {-cv_scores_mae.mean():.2f} km")
# Commentaire : La validation croisée donne une estimation plus robuste de la performance du modèle.

# --- 8. Visualisation des prédictions vs valeurs réelles (optionnel) ---
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--r', linewidth=2) # Ligne y=x
plt.title('Prédictions vs Valeurs Réelles sur l\'ensemble de test')
plt.xlabel('Valeurs Réelles (km)')
plt.ylabel('Prédictions (km)')
plt.grid(alpha=0.3)
plt.show()

print("\n--- Fin de l'analyse guidée par les instructions du projet ---")

hello world
